In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import torch
import sys
sys.path.append('..')
from shared import *
from config import CONFIG
from tqdm import tqdm_notebook as tqdm
import numpy as np

Training folder data size: 10712
Testing folder data size: 1377


In [3]:
MODEL_TO_LOAD = 'last__sampling_segmentation_D_2019-08-15_11:01:41.933482_Size_256_Ep_52_TL_0.40_VL_0.91_TM_0.63_VM_0.29.pth'
DEVICE = 'cuda'
net = torch.load(MODEL_TO_LOAD, map_location=DEVICE)

In [4]:
from torch import nn
v_cnt = 0
masks = []
preds = []
o_preds = []
imgs = []
loss_function = torch.nn.BCEWithLogitsLoss()
valid_files = []
import segmentation_models_pytorch as smp
metrics = [
        smp.utils.metrics.IoUMetric(eps=1.),
        smp.utils.metrics.FscoreMetric(eps=1.),
    ]

trainer = Trainer(net, None, loss_function, metrics, device=DEVICE)
with torch.no_grad():
               
    for v_batch in tqdm(trainer.valid_loader, desc='Performing validation'):
        v_cnt += 1
        valid_files.extend(v_batch['files'])
        out, loss, metrics = trainer.valid_step(v_batch)
        out_masks = out[-2]
        out_o_preds = out[-1]
        out_preds = nn.Sigmoid()(out[-1])
        out_imgs = out[0]
        masks.extend([v.cpu().data.numpy() for v in out_masks
                     ])
        preds.extend([v.cpu().data.numpy() for  v in out_preds])
        o_preds.extend([v.cpu().data.numpy() for  v in out_o_preds])
        imgs.extend([v.cpu().data.numpy() for v in out_imgs])
preds = np.array(preds).squeeze()
masks = np.array(masks).squeeze()


In [5]:
def np_dice_coeff(pred, target):
    smooth = 1.0
    coeff = 0
    intersection = np.sum(pred * target)
    coeff = (2 * intersection + smooth)/ (np.sum(pred + target) + smooth)
    return coeff
def get_separate_coeffs(preds, targets):
    coeffs = []
    for pred, target in zip(preds, targets):
        coeffs.append(np_dice_coeff(pred, target))
    return coeffs
coeffs = get_separate_coeffs(preds, masks)
coeffs

[0.009708718410673436,
 0.0013870388841492095,
 0.002673886539472104,
 0.003058145608316591,
 0.001557724191198993,
 0.008403386849603579,
 0.003215520359606522,
 0.9989949886190804,
 0.9992688123420189,
 0.0026247338255389343,
 0.0008264622781890857,
 0.0020790604584665596,
 0.0064103107732580355,
 0.9990429191907841,
 0.0011428923369997562,
 0.004926110641618067,
 0.002865422017785153,
 0.0029498634705946735,
 0.0012180747122793449,
 0.001366134469538377,
 0.0012772355929339242,
 0.0011601608460098326,
 0.9993773853004246,
 0.9991960226704352,
 0.9990609866520335,
 0.010526299323159936,
 0.007042305470472642,
 0.002298943355761751,
 0.002066131471677491,
 0.9990517619899779,
 0.0007107647034977154,
 0.005000008181701677,
 0.00892854314517149,
 0.004132313527857445,
 0.9989346929283335,
 0.0030303128615042246,
 0.0036901191194205246,
 0.004386029623698406,
 0.0028654221085363846,
 0.0008757554380856875,
 0.003115353623742801,
 0.9994644805266135,
 0.009259175157448353,
 0.998731618858

In [6]:
preds.max()

2.1302399e-06

In [7]:
low_thres_mes = np.linspace(0, 0.5, 100)
hi_thres_mes = np.linspace(0.1, 0.5, 100)
dice_coeff = np.zeros((100, 100))

for low_cnt, low_thres in enumerate(low_thres_mes):
    for hi_cnt, hi_thres in enumerate(low_thres_mes):
        c_preds = preds.copy()
        c_preds[compare_with_low_thres(np.sum(preds.reshape((preds.shape[0], -1)), axis=1), low_thres), :, :] = 0
        c_preds = c_preds > hi_thres
        dice_coeff[low_cnt, hi_cnt] = np_dice_coeff(c_preds, masks)

In [8]:
cpreds = preds.copy()
cpreds[cpreds>0.001] = 1
coeffs =  get_separate_coeffs(cpreds, masks)
print(np.mean(coeffs))
mat = np.vstack((np.sum(preds.reshape((cpreds.shape[0], -1))>0.001, axis=1),
                 np.sum(preds.reshape((cpreds.shape[0], -1)), axis=1), 
                 np.max(preds.reshape((cpreds.shape[0], -1)), axis=1),
                 np.sum(masks.reshape((cpreds.shape[0], -1)), axis=1),
                coeffs)).T
import pandas as pd
print(pd.DataFrame(mat))


0.499789543968583
       0         1             2       3         4
0    0.0  0.000999  4.962664e-07   102.0  0.009709
1    0.0  0.000978  6.249688e-07   720.0  0.001387
2    0.0  0.001250  3.668713e-07   373.0  0.002674
3    0.0  0.001047  5.282955e-07   326.0  0.003058
4    0.0  0.001000  3.950545e-07   641.0  0.001558
5    0.0  0.000855  3.560110e-07   118.0  0.008403
6    0.0  0.001165  5.609766e-07   310.0  0.003216
7    0.0  0.001006  4.191208e-07     0.0  0.998995
8    0.0  0.000732  3.449633e-07     0.0  0.999269
9    0.0  0.000936  3.210151e-07   380.0  0.002625
10   0.0  0.000544  2.022278e-07  1209.0  0.000826
11   0.0  0.001037  5.387728e-07   480.0  0.002079
12   0.0  0.001050  8.253695e-07   155.0  0.006410
13   0.0  0.000958  3.155410e-07     0.0  0.999043
14   0.0  0.000947  5.983872e-07   874.0  0.001143
15   0.0  0.000819  3.650258e-07   202.0  0.004926
16   0.0  0.000829  2.969380e-07   348.0  0.002865
17   0.0  0.000916  3.720985e-07   338.0  0.002950
18   0.0  0.0

In [9]:
np.mean(coeffs)

0.499789543968583

In [10]:
%matplotlib inline
import matplotlib.pyplot as plt
num = CONFIG.valid_size
import matplotlib.gridspec as gridspec
cnt = 0
r = 4
fig= plt.figure(figsize=(3 * r, num * r))
gs = gridspec.GridSpec(
        nrows=num, ncols=3, figure=fig)
# fig, axarr = plt.subplots(num, 3, gridspec_kw = {'wspace':0, 'hspace':0}, squeeze=True)

for cnt, (name, mask) in enumerate(zip(valid_files, preds)):
    axe0 = fig.add_subplot(gs[cnt, 0])
    axe1 = fig.add_subplot(gs[cnt, 1])
    axe2 = fig.add_subplot(gs[cnt, 2])
    sample = trainer.valid_dataset[name]
    im = sample['image'].cpu().data.numpy()
    true = sample['mask'].cpu().data.numpy().squeeze()
    im = np.rollaxis(im, 0, 3).squeeze()
    axe0.axis("off")
    axe0.imshow(im, aspect='equal')
    axe1.axis('off')
    axe1.imshow(true, aspect='equal')
    axe2.axis('off')
    axe2.imshow(mask.squeeze(), aspect='equal')
    cnt += 1
    if cnt == num:
        break

Error in callback <function install_repl_displayhook.<locals>.post_execute at 0x7f27bc0eda60> (for post_execute):


ValueError: Image size of 864x144000 pixels is too large. It must be less than 2^16 in each direction.

ValueError: Image size of 864x144000 pixels is too large. It must be less than 2^16 in each direction.

<Figure size 864x144000 with 1500 Axes>

In [11]:
dice_coeff[:, -1]

array([5.51304386e-06, 5.51304386e-06, 5.51304386e-06, 5.51304386e-06,
       5.51304386e-06, 5.51304386e-06, 5.51304386e-06, 5.51304386e-06,
       5.51304386e-06, 5.51304386e-06, 5.51304386e-06, 5.51304386e-06,
       5.51304386e-06, 5.51304386e-06, 5.51304386e-06, 5.51304386e-06,
       5.51304386e-06, 5.51304386e-06, 5.51304386e-06, 5.51304386e-06,
       5.51304386e-06, 5.51304386e-06, 5.51304386e-06, 5.51304386e-06,
       5.51304386e-06, 5.51304386e-06, 5.51304386e-06, 5.51304386e-06,
       5.51304386e-06, 5.51304386e-06, 5.51304386e-06, 5.51304386e-06,
       5.51304386e-06, 5.51304386e-06, 5.51304386e-06, 5.51304386e-06,
       5.51304386e-06, 5.51304386e-06, 5.51304386e-06, 5.51304386e-06,
       5.51304386e-06, 5.51304386e-06, 5.51304386e-06, 5.51304386e-06,
       5.51304386e-06, 5.51304386e-06, 5.51304386e-06, 5.51304386e-06,
       5.51304386e-06, 5.51304386e-06, 5.51304386e-06, 5.51304386e-06,
       5.51304386e-06, 5.51304386e-06, 5.51304386e-06, 5.51304386e-06,
      

In [12]:
inds = np.unravel_index(np.argmax(dice_coeff), dice_coeff.shape)
noise_th = low_thres_mes[inds[0]]
best_thr = hi_thres_mes[inds[1]]
dice_coeff[inds], noise_th, best_thr

(0.011010068928448892, 0.0, 0.1)

In [13]:
best_thr = 0.7

In [14]:
%load_ext autoreload
%autoreload 2

In [19]:
import cv2
f_names = []
rles = []
out_masks = []

zcnt = 0
cnt = 0
tester = Tester(net, device=DEVICE)
for batch in tqdm(tester.loader):
    f_names.extend(batch['files'])
    out = tester.predict(batch)
    b_rles, zeros = tester.compute_rles(out, noise_th, best_thr, keep_largest=True, h=1024, w=1024)
    zcnt +=  zeros
    rles.extend(b_rles)
print('Ratio of empty masks', zcnt / len(rles))

Training folder data size: 10712
Testing folder data size: 1377



Ratio of empty masks 1.0


In [20]:
import pandas as pd
output = pd.DataFrame({'ImageId':[os.path.splitext(f)[0] for f in f_names], 'EncodedPixels':rles})
output.to_csv('submission.csv', index=False)

In [21]:
%matplotlib inline
import matplotlib.pyplot as plt
num = 10
cnt = 0
for name, mask in zip(f_names, out_masks):
    if np.any(mask):
        fig, axes = plt.subplots(2)
        im = tester.dataset[name]['image'].cpu().data.numpy()

        im = np.rollaxis(im, 0, 3).squeeze()
        
        axes[0].imshow(cv2.resize(im, (1024, 1024)))
        axes[1].imshow(mask)
        cnt += 1
        if cnt == num:
            break

In [22]:
sample_sub = pd.read_csv('sample_submission.csv')

In [23]:
flist = list(sample_sub.ImageId)

In [24]:
np.sum([os.path.splitext(f)[0] in flist for f in f_names])

1372